In [28]:
!pip install gradio


**Prediction Code for Gradio**

Here’s the structure of the full script:
- Model loading
- Frequency encoding
- Gradio interface
- A histogram of the predicted price

In [36]:
import pandas as pd
import numpy as np
import joblib
import gradio as gr
import matplotlib.pyplot as plt
import io
from PIL import Image

# Load model and frequency encoding data
saved_model = joblib.load("lgb_model_log.pkl")
dublin_aggregated_df = pd.read_csv("dublin_aggregated_df.csv")

# Prediction function with histogram
def predict_price(
    host_id,
    host_response_rate,
    host_is_superhost,
    host_listings_count,
    accommodates,
    bathrooms,
    bedrooms,
    beds,
    avg_rating,
    number_of_reviews,
    neighbourhood_cleansed,
    property_type
):
    # Frequency encode
    neighbourhood_cleansed_freq = dublin_aggregated_df[
        dublin_aggregated_df["neighbourhood_cleansed"] == neighbourhood_cleansed
    ]["neighbourhood_cleansed_freq"].mean()

    property_type_freq = dublin_aggregated_df[
        dublin_aggregated_df["property_type"] == property_type
    ]["property_type_freq"].mean()

    # Handle missing encodings
    if pd.isna(neighbourhood_cleansed_freq):
        neighbourhood_cleansed_freq = 0
    if pd.isna(property_type_freq):
        property_type_freq = 0

    # Create input DataFrame
    property_details = pd.DataFrame({
        'host_id': [int(host_id)],
        'host_response_rate': [float(host_response_rate)],
        'host_is_superhost': [1 if host_is_superhost.lower() == 'yes' else 0],
        'host_listings_count': [int(host_listings_count)],
        'accommodates': [int(accommodates)],
        'bathrooms': [float(bathrooms)],
        'bedrooms': [float(bedrooms)],
        'beds': [float(beds)],
        'avg_rating': [float(avg_rating)],
        'number_of_reviews': [int(number_of_reviews)],
        'neighbourhood_cleansed_freq': [neighbourhood_cleansed_freq],
        'property_type_freq': [property_type_freq]
    })

    # Predict log price and convert
    log_price = saved_model.predict(property_details)
    price = np.exp(log_price[0])

    # Create histogram
    fig, ax = plt.subplots()
    ax.hist([price], bins=10, color='skyblue')
    ax.set_title("Predicted Price Histogram")
    ax.set_xlabel("Price (GBP)")
    ax.set_ylabel("Frequency")

    # Save plot to image
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    image = Image.open(buf)

    return f"Predicted Price: GBP {price:.2f}", image


**Gradio Interface**

In [37]:
interface = gr.Interface(
    fn=predict_price,
    inputs=[
        gr.Text(label="Host ID"),
        gr.Number(label="Host Response Rate (as decimal, e.g. 0.98)"),
        gr.Radio(["yes", "no"], label="Is Host a Superhost?"),
        gr.Number(label="Host Listings Count"),
        gr.Number(label="Accommodates"),
        gr.Number(label="Bathrooms"),
        gr.Number(label="Bedrooms"),
        gr.Number(label="Beds"),
        gr.Number(label="Average Rating"),
        gr.Number(label="Number of Reviews"),
        gr.Text(label="Neighbourhood"),
        gr.Text(label="Property Type")
    ],
    outputs=[
        gr.Text(label="Predicted Price"),
        gr.Image(label="Price Histogram")
    ],
    title="Airbnb Price Predictor",
    description="Enter listing details to predict nightly price in GBP."
)

interface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://198c1a1188b40e82bd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
